<a href="https://colab.research.google.com/github/MARCIANO04/Marcianosawael_TM9/blob/main/Tugas5_DeepL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 # Install ultralytics, DeepSORT, dan library lain
!pip install ultralytics opencv-python deep_sort_realtime hydra-core --upgrade --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [2]:
from google.colab import files

# Upload video file dari lokal
uploaded = files.upload()
# Pastikan file bernama: people.mp4


Saving people.mp4 to people.mp4


In [4]:
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import os

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # Versi ringan, bisa ganti ke yolov8s.pt untuk akurasi lebih tinggi

# Inisialisasi DeepSORT tracker
tracker = DeepSort(max_age=30)

# Buka video
video_path = "people.mp4"
cap = cv2.VideoCapture(video_path)

# Simpan output
output_path = "hasil_masuk_keluar.avi"
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, 20.0, (640, 480))

# Konfigurasi garis & hitungan
line_y = 240  # Garis horizontal tengah
masuk, keluar = 0, 0
counted_ids = set()
track_history = {}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640, 480))
    results = model(frame)[0]
    detections = []

    # Deteksi orang saja
    for r in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = r
        if int(class_id) == 0:  # 'person'
            detections.append(([x1, y1, x2 - x1, y2 - y1], score, 'person'))

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)

        # Simpan history lintasan
        if track_id not in track_history:
            track_history[track_id] = []
        track_history[track_id].append((cx, cy))

        # Gambar kotak & ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID:{track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

        # Hitung masuk/keluar
        if len(track_history[track_id]) >= 2:
            y_prev = track_history[track_id][-2][1]
            y_curr = track_history[track_id][-1][1]

            if track_id not in counted_ids:
                # Keluar (dari bawah ke atas)
                if y_prev > line_y and y_curr <= line_y:
                    keluar += 1
                    counted_ids.add(track_id)
                # Masuk (atas ke bawah)
                elif y_prev < line_y and y_curr >= line_y:
                    masuk += 1
                    counted_ids.add(track_id)

    # Gambar garis dan jumlah
    cv2.line(frame, (0, line_y), (640, line_y), (0, 0, 255), 2)
    cv2.putText(frame, f'Masuk: {masuk}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(frame, f'Keluar: {keluar}', (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 255), 2)

    out.write(frame)

cap.release()
out.release()



0: 480x640 4 persons, 2 handbags, 207.0ms
Speed: 5.1ms preprocess, 207.0ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 2 handbags, 184.4ms
Speed: 2.4ms preprocess, 184.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 2 handbags, 224.4ms
Speed: 3.5ms preprocess, 224.4ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 2 handbags, 185.6ms
Speed: 4.3ms preprocess, 185.6ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 2 handbags, 182.5ms
Speed: 2.5ms preprocess, 182.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 2 handbags, 204.7ms
Speed: 3.9ms preprocess, 204.7ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 2 handbags, 203.0ms
Speed: 5.8ms preprocess, 203.0ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4

In [5]:
from google.colab import files
files.download("hasil_masuk_keluar.avi")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>